<a href="https://colab.research.google.com/github/eomtaehyeon/Hongong_ML-DL-/blob/main/Chapter_5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 교차 검증과 그리드 서치
- 키워드 : 하이퍼 파라미터 (그리드 서치 vs 랜덤서치)
- 데이터가 작을 때, 주로 사용
- 하이퍼파라미터
  + max_depth : 3, 정확도가 84%
- 결론
  + 모르면 디폴트만 쓰자!
  + 가성비(시간 대비 성능 보장 안됨!)

# 검증세트
- 테스트 세트를 사용하지 않고 이를 측정하는 간단한 방법은 훈련세트를 또 나눈 데이터
- 테스트 세트 (1회성)
- 훈련데이터를 훈련 데이터 + 검증 데이터로 재 분할

## 현실
- 테스트 데이터가 별도로 존재하지 않음!
- 전체 데이터 = 훈련(6) : 검증(2) : 테스트(2)
  + 테스트 데이터는 모르는 데이터로 생각!

In [ ]:
# 데이터 불러오기
import pandas as pd
wine = pd.read_csv('http://bit.ly/wine_csv_data')

data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

# 훈련세트와 테스트세트로 나눔.
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42
)
# 훈련세트와 검증세트를 만듦
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42
)

# 훈련세트 검증세트의 크기
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


## 모델 만든 후 평가
- 과대적합 발생

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


## 교차 검증
- 검증 세트를 떼어 내어 평가하는 과정을 여러 번 반복.
  + 3-폴드 교차 검증 : 훈련 세트를 세 부분으로 나눠서 교차 검증을 수행하는 것.
  + 보통 5-폴드 교차 검증이나 10-폴드 교차 검증을 많이 씀.
- cross_validate()함수 사용해서 교차 검증.
  + 사이킷런에는 cross_validate() 함수의 전신인 cross_val_score()함수도 있다. 이 함수는 cross_validate() 함수의 결과 중에서 test_score 값만 반환.
  + 이 함수는 fit_time, score_time, test_score 키를 가진 딕셔너리를 반환.
  + 기본적으로 5-폴드 교차 검증을 수행.
- 교차 검증의 목적 : 좋은 모델이 만들어진다!
  + 좋은 모델 != 성능 좋은 모델
  + 좋은 모델 = 과대적합이 아닌 모델 = 모형의 오차가 적은 모델 = 안정적인 모델
- 단점 : 시간이 오래 걸림


# 교차 검증 함수

In [ ]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.01392674, 0.01075053, 0.01143503, 0.01178861, 0.01016068]), 'score_time': array([0.00142336, 0.001369  , 0.00134206, 0.00183415, 0.00129271]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


- fit_time, score_time은 각각 모델을 훈련하는 시간과 검증하는 시간을 의미.
- 교차 검증의 최종 점수는 test_score 5개의 점수를 평균.

In [ ]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


  + 주의점 : cross_validate()는 훈련 세트를 섞어 폴드를 나누지 않는다. 교차 검증을 할 때 훈련 세트를 섞으려면 분할기를 지정해야 한다.
  + 회귀 모델 : KFold, 분류 모델 : StratifiedKFold

In [ ]:
# StratifiedKFold 사용
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv = StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


- 훈련 세트 섞은 후 10-폴드 교차검증

In [ ]:
from sklearn.model_selection import StratifiedKFold
splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
scores = cross_validate(dt, train_input, train_target, cv = splitter)

print(np.mean(scores['test_score']))

0.8574181117533719


## 하이퍼파라미터 튜닝 꼭 하고 싶다!
- 랜덤 서치 사용하자!
- 자동으로 잡아주는 라이브러리들이 등장하기 시작함
  + hyperopt
- 그리드 서치는 잘모르기때문에 비추

In [ ]:
%%time

from sklearn.model_selection import GridSearchCV
params = {
    'min_impurity_decrease' :[0.0001, 0.0002, 0.0003, 0.0004, 0.0005]
          }


# dt = DecisionTreeClassifier(random_state=42)
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1) # -1은 모든 코어를 다 쓰겠다.
gs.fit(train_input, train_target)

CPU times: user 76.3 ms, sys: 656 µs, total: 76.9 ms
Wall time: 178 ms


In [ ]:
# max_depth 값 추가
"""
%%time

from sklearn.model_selection import GridSearchCV
params = {
    'min_impurity_decrease' :[0.0001, 0.0002, 0.0003, 0.0004, 0.0005],
    'max_depth' : [3,4,5,6,7]
          }


# dt = DecisionTreeClassifier(random_state=42)
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1) # -1은 모든 코어를 다 쓰겠다.
gs.fit(train_input, train_target)
"""

CPU times: user 190 ms, sys: 5.64 ms, total: 196 ms
Wall time: 934 ms


In [ ]:
dt = gs.best_estimator_
print(dt)
print(dt.score(train_input, train_target))
print(gs.best_params_)

DecisionTreeClassifier(min_impurity_decrease=0.0001, random_state=42)
0.9615162593804117
{'min_impurity_decrease': 0.0001}


In [ ]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


# 랜덤 서치
- 매개변수 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링할 수 있도록 확률 분포 객체를 전달.


In [ ]:
from scipy.stats import uniform, randint
rgen = randint(0, 10)
rgen.rvs(10)

array([0, 7, 1, 8, 7, 5, 1, 6, 8, 3])

In [ ]:
np.unique(rgen.rvs(1000), return_counts = True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 85, 113, 118, 105,  93,  86, 110,  93,  96, 101]))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# p.254
params = {
    'min_impurity_decrease' : uniform(0.0001, 0.001),
    'max_depth': randint(20, 50),
    'min_samples_split' : randint(2, 25),
    'min_samples_leaf' : randint(1, 25),
}
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)

gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fd6be9fec90>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fd6be0d64d0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fd6be0f7cd0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fd6be0d1990>},
                   random_state=42)

In [ ]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [ ]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86


# References : 
- 혼자 공부하는 머신러닝+딥러닝 p. 242 ~ 262
- Hyperparameter (machine learning) - Wikipedia : https://en.wikipedia.org/wiki/Hyperparameter_(machine_learning)
- RandomizedSearchCV : https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html